# Getting Started with Azure Quantum Resource Estimation using Qiskit

👋 Welcome to the Azure Quantum Resource Estimator. In this notebook we will
guide you how to estimate and analyze the physical resource estimates of a
quantum program targeted for execution based on the architecture design of a
fault-tolerant quantum computer. As a running example we are using a multiplier.

## Implementing the algorithm

As a first step, we will create a sample application which will be used throughout this Resource Estimation notebook. To start, we'll import some Python packages from `azure.quantum` and `qiskit`.

In [1]:
# These are the Python imports that we use in this Qiskit-based example

from azure.quantum.qiskit import AzureQuantumProvider
from qiskit import QuantumCircuit, transpile
from qiskit.circuit.library import RGQFTMultiplier


We are creating a quantum circuit for a multiplier based on the construction presented in [arXiv:1411.5949](https://arxiv.org/abs/1411.5949) which uses the Quantum Fourier Transform to implement arithmetic. You can adjust the size of the multiplier by changing the `bitwidth` variable. The circuit generation is wrapped in a function that can be called with the bitwidth of the multiplier. The circuit will have two input registers with that bitwidth, and one output register with the size of twice the bitwidth. The function will also print some logical resource counts for the multiplier extracted directly from the quantum circuit.

In [2]:
def create_algorithm(bitwidth, backend):
    print(f"[INFO] Create a QFT-based multiplier with bitwidth {bitwidth}")
    
    # Print a warning for large bitwidths that will require some time to generate and
    # transpile the circuit.
    if bitwidth > 18:
        print(f"[WARN] It will take more than one minute generate a quantum circuit with a bitwidth larger than 18")

    circ = RGQFTMultiplier(num_state_qubits=bitwidth, num_result_qubits=2 * bitwidth)

    # One could further reduce the resource estimates by increasing the optimization_level,
    # however, this will also increase the runtime to construct the algorithm.  Note, that
    # it does not affect the runtime for resource estimation.
    print(f"[INFO] Decompose circuit into intrinsic quantum operations")

    # retrieve basis gates from backend
    basis_gates = backend.configuration().basis_gates
    circ = transpile(circ, basis_gates=basis_gates, optimization_level=0)

    # print some statistics
    print(f"[INFO]   qubit count: {circ.num_qubits}")
    print("[INFO]   gate counts")
    for gate, count in circ.count_ops().items():
        print(f"[INFO]   - {gate}: {count}")

    return circ


## Estimating the algorithm

Let's connect to the Azure Quantum workspace and create a backend instance for
the Azure Quantum Resource Estimator.  The backend acts as a target for our
quantum computing jobs.  Examples for other backends include QPUs to execute
quantum programs on today's quantum computers, or simulators to simulate the
functional behavior of a quantum program.  You can find examples for such
backends in other notebooks in the _Sample Gallery_.

In [3]:
from azure.quantum import Workspace
from azure.quantum.qiskit import AzureQuantumProvider

workspace = Workspace(
            resource_id = "/subscriptions/2f90c9d6-c623-42f8-82f7-a36891f2a93f/resourceGroups/AzureQuantum/providers/Microsoft.Quantum/Workspaces/wigners-friend2",
            location = "westeurope")


provider = AzureQuantumProvider(workspace)


In [5]:
backend = provider.get_backend('microsoft.estimator')


Next we will create an instance of our algorithm using the `create_algorithm` function. You can adjust the size of the multiplier by changing the `bitwidth` variable.

In [6]:
bitwidth = 4

circ = create_algorithm(bitwidth, backend)


[INFO] Create a QFT-based multiplier with bitwidth 4
[INFO] Decompose circuit into intrinsic quantum operations
[INFO]   qubit count: 16
[INFO]   gate counts
[INFO]   - rz: 1320
[INFO]   - cx: 1136
[INFO]   - h: 16


Let's now estimate the physical resources for this circuit using the default assumptions. We can submit the circuit to the Resource Estimation backend using its `run` method.

In [7]:
job = backend.run(circ)
result = job.result()


........

The simplest way to inspect the results of the job is to output them to the notebook. This will output a table with the overall physical resource counts. You can further inspect more details about the resource estimates by collapsing various groups which have more information. For example, if you collapse the *Logical qubit parameters* group, you can see that the quantum error correction (QEC) code distance is 15. In the last group you can see the physical qubit properties that were assumed for this estimation. For example, we see that the time to perform a single-qubit measurement and a single-qubit gate are assumed to be 100 ns and 50 ns, respectively.

In [8]:
result


Runtime,37 millisecs,"Total runtime This is a runtime estimate for the execution time of the algorithm. In general, the execution time corresponds to the duration of one logical cycle (6,000 nanosecs) multiplied by the 6,209 logical cycles to run the algorithm. If however the duration of a single T factory (here: 83,200 nanosecs) is larger than the algorithm runtime, we extend the number of logical cycles artificially in order to exceed the runtime of a single T factory."
rQOPS,7.50M,"Reliable quantum operations per second The value is computed as the number of logical qubits after layout (45) (with a logical error rate of 1.19e-9) multiplied by the clock frequency (166,666.67), which is the number of logical cycles per second."
Physical qubits,524.25k,"Number of physical qubits This value represents the total number of physical qubits, which is the sum of 20,250 physical qubits to implement the algorithm logic, and 504,000 physical qubits to execute the T factories that are responsible to produce the T states that are consumed by the algorithm."
Logical algorithmic qubits,45,"Number of logical qubits for the algorithm after layout Laying out the logical qubits in the presence of nearest-neighbor constraints requires additional logical qubits. In particular, to layout the $Q_{\rm alg} = 16$ logical qubits in the input algorithm, we require in total $2 \cdot Q_{\rm alg} + \lceil \sqrt{8 \cdot Q_{\rm alg}}\rceil + 1 = 45$ logical qubits."
Algorithmic depth,6.21k,"Number of logical cycles for the algorithm To execute the algorithm using Parallel Synthesis Sequential Pauli Computation (PSSPC), operations are scheduled in terms of multi-qubit Pauli measurements, for which assume an execution time of one logical cycle. Based on the input algorithm, we require one multi-qubit measurement for the 0 single-qubit measurements, the 702 arbitrary single-qubit rotations, and the 186 T gates, three multi-qubit measurements for each of the 0 CCZ and 0 CCiX gates in the input program, as well as 17 multi-qubit measurements for each of the 313 non-Clifford layers in which there is at least one single-qubit rotation with an arbitrary angle rotation."
Logical depth,6.21k,"Number of logical cycles performed This number is usually equal to the logical depth of the algorithm, which is 6,209. However, in the case in which a single T factory is slower than the execution time of the algorithm, we adjust the logical cycle depth to exceed the T factory's execution time."
Clock frequency,166.67k,Number of logical cycles per second This is the number of logical cycles that can be performed within one second. The logical cycle time is 6 microsecs.
Number of T states,12.12k,"Number of T states consumed by the algorithm To execute the algorithm, we require one T state for each of the 186 T gates, four T states for each of the 0 CCZ and 0 CCiX gates, as well as 17 for each of the 702 single-qubit rotation gates with arbitrary angle rotation."
Number of T factories,28,"Number of T factories capable of producing the demanded 12,120 T states during the algorithm's runtime The total number of T factories 28 that are executed in parallel is computed as $\left\lceil\dfrac{\text{T states}\cdot\text{T factory duration}}{\text{T states per T factory}\cdot\text{algorithm runtime}}\right\rceil = \left\lceil\dfrac{12,120 \cdot 83,200\;\text{ns}}{1 \cdot 37,254,000\;\text{ns}}\right\rceil$"
Number of T factory invocations,433,"Number of times all T factories are invoked In order to prepare the 12,120 T states, the 28 copies of the T factory are repeatedly invoked 433 times."
Physical algorithmic qubits,20.25k,"Number of physical qubits for the algorithm after layout The 20,250 are the product of the 45 logical qubits after layout and the 450 physical qubits that encode a single logical qubit."


The distribution of physical qubits used for the execution of the algorithm instructions and the supporting T factories can provide us valuable information to guide us in applying space and time optimizations. We can visualize this distribution to better understand the estimated space requirements for our algorithm.

In [8]:
result.diagram.space


We can also visualize the time required to execute the algorithm as it relates to each T factory invocation runtime and the number of T factory invocations.

>
>  *You cannot visualize the time and space diagrams in the same cell.*
>

In [10]:
result.diagram.time


If you prefer a more compact version of the table, in which the descriptions are
provided by means of tooltips, you can write:

In [11]:
result.summary


"This is a runtime estimate for the execution time of the algorithm. In general, the execution time corresponds to the duration of one logical cycle (6,000 nanosecs) multiplied by the 6,209 logical cycles to run the algorithm. If however the duration of a single T factory (here: 83,200 nanosecs) is larger than the algorithm runtime, we extend the number of logical cycles artificially in order to exceed the runtime of a single T factory.Runtime",37 millisecs,Total runtime
"The value is computed as the number of logical qubits after layout (45) (with a logical error rate of 1.19e-9) multiplied by the clock frequency (166,666.67), which is the number of logical cycles per second.rQOPS",7.50M,Reliable quantum operations per second
"This value represents the total number of physical qubits, which is the sum of 20,250 physical qubits to implement the algorithm logic, and 504,000 physical qubits to execute the T factories that are responsible to produce the T states that are consumed by the algorithm.Physical qubits",524.25k,Number of physical qubits
"Laying out the logical qubits in the presence of nearest-neighbor constraints requires additional logical qubits. In particular, to layout the $Q_{\rm alg} = 16$ logical qubits in the input algorithm, we require in total $2 \cdot Q_{\rm alg} + \lceil \sqrt{8 \cdot Q_{\rm alg}}\rceil + 1 = 45$ logical qubits.Logical algorithmic qubits",45,Number of logical qubits for the algorithm after layout
"To execute the algorithm using Parallel Synthesis Sequential Pauli Computation (PSSPC), operations are scheduled in terms of multi-qubit Pauli measurements, for which assume an execution time of one logical cycle. Based on the input algorithm, we require one multi-qubit measurement for the 0 single-qubit measurements, the 702 arbitrary single-qubit rotations, and the 186 T gates, three multi-qubit measurements for each of the 0 CCZ and 0 CCiX gates in the input program, as well as 17 multi-qubit measurements for each of the 313 non-Clifford layers in which there is at least one single-qubit rotation with an arbitrary angle rotation.Algorithmic depth",6.21k,Number of logical cycles for the algorithm
"This number is usually equal to the logical depth of the algorithm, which is 6,209. However, in the case in which a single T factory is slower than the execution time of the algorithm, we adjust the logical cycle depth to exceed the T factory's execution time.Logical depth",6.21k,Number of logical cycles performed
This is the number of logical cycles that can be performed within one second. The logical cycle time is 6 microsecs.Clock frequency,166.67k,Number of logical cycles per second
"To execute the algorithm, we require one T state for each of the 186 T gates, four T states for each of the 0 CCZ and 0 CCiX gates, as well as 17 for each of the 702 single-qubit rotation gates with arbitrary angle rotation.Number of T states",12.12k,Number of T states consumed by the algorithm
"The total number of T factories 28 that are executed in parallel is computed as $\left\lceil\dfrac{\text{T states}\cdot\text{T factory duration}}{\text{T states per T factory}\cdot\text{algorithm runtime}}\right\rceil = \left\lceil\dfrac{12,120 \cdot 83,200\;\text{ns}}{1 \cdot 37,254,000\;\text{ns}}\right\rceil$Number of T factories",28,"Number of T factories capable of producing the demanded 12,120 T states during the algorithm's runtime"
"In order to prepare the 12,120 T states, the 28 copies of the T factory are repeatedly invoked 433 times.Number of T factory invocations",433,Number of times all T factories are invoked
"The 20,250 are the product of the 45 logical qubits after layout and the 450 physical qubits that encode a single logical qubit.Physical algorithmic qubits",20.25k,Number of physical qubits for the algorithm after layout


We can also programmatically access all the values that can be passed to the job execution and see which default values were assumed:

In [12]:
result.data()["jobParams"]


{'qecScheme': {'name': 'surface_code',
  'errorCorrectionThreshold': 0.01,
  'crossingPrefactor': 0.03,
  'logicalCycleTime': '(4 * twoQubitGateTime + 2 * oneQubitMeasurementTime) * codeDistance',
  'physicalQubitsPerLogicalQubit': '2 * codeDistance * codeDistance',
  'maxCodeDistance': 50},
 'errorBudget': 0.001,
 'qubitParams': {'instructionSet': 'GateBased',
  'name': 'qubit_gate_ns_e3',
  'oneQubitMeasurementTime': '100 ns',
  'oneQubitGateTime': '50 ns',
  'twoQubitGateTime': '50 ns',
  'tGateTime': '50 ns',
  'oneQubitMeasurementErrorRate': 0.001,
  'oneQubitGateErrorRate': 0.001,
  'twoQubitGateErrorRate': 0.001,
  'tGateErrorRate': 0.001,
  'idleErrorRate': 0.001}}

We see that there are three input parameters that can be customized: `qubitParams`, `qecScheme`, and `errorBudget`.

### Qubit parameters

The first parameter `qubitParams` is used to specify qubit parameters.  When
modeling the physical qubit abstractions, we distinguish between two different
physical instruction sets that are used to operate the qubits.  The physical
instruction set can be either *gate-based* or *Majorana*.  A gate-based
instruction set provides single-qubit measurement, single-qubit gates (incl. T
 gates), and two-qubit gates.  A Majorana instruction set provides a physical T
 gate, single-qubit measurement and two-qubit joint measurement operations.

Qubit parameters can be completely customized.  Before we show this, we show hot
to choose from six pre-defined qubit parameters, four of which have gate-based
instruction sets and two with a Majorana instruction set.  An overview of all
pre-defined qubit parameters is provided by the following table:

| Pre-defined qubit parameters | Instruction set | References                                                                                                 |
|------------------------------|-----------------|------------------------------------------------------------------------------------------------------------|
| `"qubit_gate_ns_e3"`         | gate-based      | [arXiv:2003.00024](https://arxiv.org/abs/2003.00024), [arXiv:2111.11937](https://arxiv.org/abs/2111.11937) |
| `"qubit_gate_ns_e4"`         | gate-based      | [arXiv:2003.00024](https://arxiv.org/abs/2003.00024), [arXiv:2111.11937](https://arxiv.org/abs/2111.11937) |
| `"qubit_gate_us_e3"`         | gate-based      | [arXiv:1701.04195](https://arxiv.org/abs/1701.04195)                                                       |
| `"qubit_gate_us_e4"`         | gate-based      | [arXiv:1701.04195](https://arxiv.org/abs/1701.04195)                                                       |
| `"qubit_maj_ns_e4"`          | Majorana        | [arXiv:1610.05289](https://arxiv.org/abs/1610.05289)                                                       |
| `"qubit_maj_ns_e6"`          | Majorana        | [arXiv:1610.05289](https://arxiv.org/abs/1610.05289)                                                       |

Pre-defined qubit parameters can be selected by specifying the `name` field in
the `qubitParams`.  If no value is provided, `"qubit_gate_ns_e3"` is chosen as
the default qubit parameters.

Let's re-run resource estimation for our running example on the Majorana-based
qubit parameters `"qubit_maj_ns_e6"`.

In [13]:
job = backend.run(circ,
    qubitParams={
        "name": "qubit_maj_ns_e6"
    })
result = job.result()
result


......

Runtime,62 millisecs,"Total runtime This is a runtime estimate for the execution time of the algorithm. In general, the execution time corresponds to the duration of one logical cycle (10,000 nanosecs) multiplied by the 6,209 logical cycles to run the algorithm. If however the duration of a single T factory (here: 132,400 nanosecs) is larger than the algorithm runtime, we extend the number of logical cycles artificially in order to exceed the runtime of a single T factory."
rQOPS,4.50M,"Reliable quantum operations per second The value is computed as the number of logical qubits after layout (45) (with a logical error rate of 1.19e-9) multiplied by the clock frequency (100,000.00), which is the number of logical cycles per second."
Physical qubits,28.25k,"Number of physical qubits This value represents the total number of physical qubits, which is the sum of 2,250 physical qubits to implement the algorithm logic, and 26,000 physical qubits to execute the T factories that are responsible to produce the T states that are consumed by the algorithm."
Logical algorithmic qubits,45,"Number of logical qubits for the algorithm after layout Laying out the logical qubits in the presence of nearest-neighbor constraints requires additional logical qubits. In particular, to layout the $Q_{\rm alg} = 16$ logical qubits in the input algorithm, we require in total $2 \cdot Q_{\rm alg} + \lceil \sqrt{8 \cdot Q_{\rm alg}}\rceil + 1 = 45$ logical qubits."
Algorithmic depth,6.21k,"Number of logical cycles for the algorithm To execute the algorithm using Parallel Synthesis Sequential Pauli Computation (PSSPC), operations are scheduled in terms of multi-qubit Pauli measurements, for which assume an execution time of one logical cycle. Based on the input algorithm, we require one multi-qubit measurement for the 0 single-qubit measurements, the 702 arbitrary single-qubit rotations, and the 186 T gates, three multi-qubit measurements for each of the 0 CCZ and 0 CCiX gates in the input program, as well as 17 multi-qubit measurements for each of the 313 non-Clifford layers in which there is at least one single-qubit rotation with an arbitrary angle rotation."
Logical depth,6.21k,"Number of logical cycles performed This number is usually equal to the logical depth of the algorithm, which is 6,209. However, in the case in which a single T factory is slower than the execution time of the algorithm, we adjust the logical cycle depth to exceed the T factory's execution time."
Clock frequency,100.00k,Number of logical cycles per second This is the number of logical cycles that can be performed within one second. The logical cycle time is 10 microsecs.
Number of T states,12.12k,"Number of T states consumed by the algorithm To execute the algorithm, we require one T state for each of the 186 T gates, four T states for each of the 0 CCZ and 0 CCiX gates, as well as 17 for each of the 702 single-qubit rotation gates with arbitrary angle rotation."
Number of T factories,26,"Number of T factories capable of producing the demanded 12,120 T states during the algorithm's runtime The total number of T factories 26 that are executed in parallel is computed as $\left\lceil\dfrac{\text{T states}\cdot\text{T factory duration}}{\text{T states per T factory}\cdot\text{algorithm runtime}}\right\rceil = \left\lceil\dfrac{12,120 \cdot 132,400\;\text{ns}}{1 \cdot 62,090,000\;\text{ns}}\right\rceil$"
Number of T factory invocations,467,"Number of times all T factories are invoked In order to prepare the 12,120 T states, the 26 copies of the T factory are repeatedly invoked 467 times."
Physical algorithmic qubits,2.25k,"Number of physical qubits for the algorithm after layout The 2,250 are the product of the 45 logical qubits after layout and the 50 physical qubits that encode a single logical qubit."


Let's inspect the physical counts programmatically. For example, we can show all physical resource estimates and their breakdown using the `physicalCounts` field in the result data. This will show the logical qubit error and logical T-state error rates required to match the error budget. By default runtimes are shown in nanoseconds.

In [14]:
result.data()["physicalCounts"]


{'physicalQubits': 28250,
 'runtime': 62090000,
 'rqops': 4500000,
 'breakdown': {'algorithmicLogicalQubits': 45,
  'algorithmicLogicalDepth': 6209,
  'logicalDepth': 6209,
  'numTstates': 12120,
  'clockFrequency': 100000.0,
  'numTfactories': 26,
  'numTfactoryRuns': 467,
  'physicalQubitsForTfactories': 26000,
  'physicalQubitsForAlgorithm': 2250,
  'requiredLogicalQubitErrorRate': 1.1930113395727827e-09,
  'requiredLogicalTstateErrorRate': 2.7502750275027503e-08,
  'numTsPerRotation': 17,
  'cliffordErrorRate': 1e-06}}

We can also explore details about the T factory that was created to execute this algorithm.

In [15]:
result.data()["tfactory"]


{'physicalQubits': 1000,
 'runtime': 132400,
 'numTstates': 1,
 'numInputTstates': 345,
 'numRounds': 2,
 'numUnitsPerRound': [23, 1],
 'unitNamePerRound': ['15-to-1 RM prep', '15-to-1 space efficient'],
 'codeDistancePerRound': [1, 5],
 'physicalQubitsPerRound': [713, 1000],
 'runtimePerRound': [2400, 130000],
 'logicalErrorRate': 1.7090794243129628e-10}

Next, we are using this data to produce some explanations of how the T factories produce the required T states.

In [16]:
data = result.data()
tfactory = data["tfactory"]
breakdown = data["physicalCounts"]["breakdown"]
producedTstates = breakdown["numTfactories"] * breakdown["numTfactoryRuns"] * tfactory["numTstates"]

print(f"""A single T factory produces {tfactory["logicalErrorRate"]:.2e} T states with an error rate of (required T state error rate is {breakdown["requiredLogicalTstateErrorRate"]:.2e}).""")
print(f"""{breakdown["numTfactories"]} copie(s) of a T factory are executed {breakdown["numTfactoryRuns"]} time(s) to produce {producedTstates} T states ({breakdown["numTstates"]} are required by the algorithm).""")
print(f"""A single T factory is composed of {tfactory["numRounds"]} rounds of distillation:""")
for round in range(tfactory["numRounds"]):
    print(f"""- {tfactory["numUnitsPerRound"][round]} {tfactory["unitNamePerRound"][round]} unit(s)""")


A single T factory produces 1.71e-10 T states with an error rate of (required T state error rate is 2.75e-08).
26 copie(s) of a T factory are executed 467 time(s) to produce 12142 T states (12120 are required by the algorithm).
A single T factory is composed of 2 rounds of distillation:
- 23 15-to-1 RM prep unit(s)
- 1 15-to-1 space efficient unit(s)


Custom qubit parameters must completely specify all required parameters.  These are the values that are
considered when the `instructionSet` is `"GateBased"`.

| Field (*required)               | Description                                                          |
|---------------------------------|----------------------------------------------------------------------|
| `name`                          | Some descriptive name for the parameters                             |
| `oneQubitMeasurementTime`*      | Operation time for single-qubit measurement ($t_{\rm meas}$) in ns   |
| `oneQubitGateTime`*             | Operation time for single-qubit Clifford gate ($t_{\rm gate}$) in ns |
| `twoQubitGateTime`              | Operation time for two-qubit Clifford gate in ns                     |
| `tGateTime`                     | Operation time for single-qubit non-Clifford gate in ns              |
| `oneQubitMeasurementErrorRate`* | Error rate for single-qubit measurement                              |
| `oneQubitGateErrorRate`*        | Error rate for single-qubit Clifford gate ($p$)                      |
| `twoQubitGateErrorRate`         | Error rate for two-qubit Clifford gate                               |
| `tGateErrorRate`                | Error rate to prepare single-qubit non-Clifford state ($p_T$)        |

The values for `twoQubitGateTime` and `tGateTime` default to `oneQubitGateTime`
when not specified; the values for `twoQubitGateErrorRate` and `tGateErrorRate`
default to `oneQubitGateErrorRate` when not specified.

A minimum template for qubit parameters based on a gate-based instruction set
with all required values is:

```json
{
    "qubitParams": {
        "instructionSet": "GateBased",
        "oneQubitMeasurementTime": <time string>,
        "oneQubitGateTime": <time string>,
        "oneQubitMeasurementErrorRate": <double>,
        "oneQubitGateErrorRate": <double>
    }
}
```

For time units, you need to specify time strings which are double-precision
floating point numbers followed by a space and a unit prefix which is `ns`, `µs`
(alternatively `us`), `ms`, or `s`.

These are the values that are considered when the `instructionSet` is
`"Majorana"`.

| Field (*required)                   | Description                                                         |
|-------------------------------------|---------------------------------------------------------------------|
| `name`                              | Some descriptive name for the parameters                            |
| `oneQubitMeasurementTime`*          | Operation time for single-qubit measurement ($t_{\rm meas}$) in ns  |
| `twoQubitJointMeasurementTime`      | Operation time for two-qubit joint measurement in ns                |
| `tGateTime`                         | Operation time for single-qubit non-Clifford gate in ns             |
| `oneQubitMeasurementErrorRate`*     | Error rate for single-qubit measurement                             |
| `twoQubitJointMeasurementErrorRate` | Error rate for two-qubit joint measurement                          |
| `tGateErrorRate`*                   | Error rate to prepare single-qubit non-Clifford state ($p_T$)       |

The values for `twoQubitJointMeasurementTime` and `tGateTime` default to
`oneQubitGateTime` when not specified; the value for
`twoQubitJointMeasurementErrorRate` defaults to `oneQubitMeasurementErrorRate`
when not specified.

A minimum template for qubit parameters based on a Majorana instruction set with
all required values is:

```json
{
    "qubitParams": {
        "instructionSet": "Majorana",
        "oneQubitMeasurementTime": <time string>,
        "oneQubitMeasurementErrorRate": <double>,
        "tGateErrorRate": <double>
    }
}
```

### QEC schemes

To execute practical-scale quantum applications, we require operations with very
low error rates. These error rate targets are typically beyond the capabilities
of raw physical qubits. To overcome this limitation, quantum error correction
(QEC) and fault-tolerant computation are two crucial techniques that form the
building blocks of large-scale quantum computers. First, QEC allows us to
compose multiple error-prone physical qubits and build a more reliable logical
qubit that preserves quantum information better than the underlying physical
qubits. Several QEC schemes have been developed since the last three decades,
including popular schemes such as the Shor-code, surface code, color codes and
others, and recently the [Hastings-Haah code](https://arxiv.org/abs/2107.02194).
These schemes vary based on the number of physical qubits they require, the
connectivity among qubits and other factors. By using QEC techniques, we can
achieve a fault-tolerant quantum computation, enabling reliable storing and
processing of quantum information in the presence of noise. To store information
reliably, we require that the QEC scheme is able to suppress errors when the
physical qubits meet a certain threshold error rate. To process information, we
require fault-tolerant operations that allow applications to perform general
purpose quantum computations efficiently and limit the spread of errors that
occur while computing with logical qubits. Schemes for fault-tolerant operations
include techniques such as lattice surgery and transversal operations. Together,
QEC and fault-tolerance techniques bridges the accuracy gap between quantum
hardware and algorithms.

The error correction code distance (or just code distance in short) is a
parameter that controls the number of errors that can be corrected, and thus the
error rate of the logical qubits and the number of physical qubits required to
encode them.  The higher the code distance, the better the accuracy, but also
the higher the amount of physical qubits.  The goal is to find the minimum code
distance that can achieve the required error rate set for a particular
application.  We will explain later in this notebook how a global error budget
is provided as input and how it is distributed throughout the estimation,
including the logical error rate of logical qubits.

We follow the standard way of modeling logical error rates using an exponential
model parameterized by the code distance $d$, physical error rate $p$, QEC
threshold $p^*$.  The physical error rate $p$ is extracted from the qubit
parameters above as the worst-case error rate any physical Clifford operation in
the device.  In particular, we set $p = {}$ `max(oneQubitMeasurementErrorRate,
oneQubitGateErrorRate, twoQubitGateErrorRate)` for qubit parameters with a
gate-based instruction set, and $p = {}$ `max(oneQubitMeasurementErrorRate,
twoQubitJointMeasurementErrorRate)` for qubit parameters with a Majorana
instruction set.  QEC schemes typically have a error rate threshold $p^*$ below
which error correction suppresses errors.

Our current implementation uses the formula

$$
P = a\left(\frac{p}{p^*}\right)^{\frac{d+1}{2}}
$$

as the generic model.  The exact parameters for each pre-defined QEC scheme
(including  a crossing pre-factor $a$ which can be extracted numerically for
simulations) are listed below.

In Azure Quantum Resource Estimation we can abstract the quantum error
correction scheme based on the above formula by providing values for the
crossing pre-factor $a$ and the error correction threshold $p^*$.  Further, one
needs to specify the logical cycle time, i.e., the time to execute a single
logical operation, which depends on the code distance and the  physical
operation time assumptions of the underlying physical qubits.  Finally, a second
formula computes the number of physical qubits required to encode one logical
qubit based on the code distance.

As with the physical qubit parameters, one can choose from several pre-defined
QEC schemes, can extend pre-defined ones, and can provide custom schemes by
providing all parameters.  Note that QEC schemes are tightly connected to the
physical instruction set of the physical qubit parameters, and therefore are
defined specifically for one of the two instruction sets.

We provide three pre-defined QEC schemes, two `"surface_code"` protocols for
gate-based and Majorana physical instruction sets, and the `"floquet_code"`
protocol that is so far only implemented for a Majorana physical instruction set
in the resource estimator.

| QEC scheme     | Instruction set | References                                                                                                 |
|----------------|-----------------|------------------------------------------------------------------------------------------------------------|
| `surface_code` | gate-based      | [arXiv:1208.0928](https://arxiv.org/abs/1208.0928), [arXiv:1009.3686](https://arxiv.org/abs/1009.3686)     |
| `surface_code` | Majorana        | [arXiv:1909.03002](https://arxiv.org/abs/1909.03002), [arXiv:2007.00307](https://arxiv.org/abs/2007.00307) |
| `floquet_code` | Majorana        | [arXiv:2202.11829](https://arxiv.org/abs/2202.11829)                                                       |

In case of `"surface_code"` the corresponding scheme is selected based on the
qubit type of the physical qubit parameters.  The gate-based surface code is
based on [[arXiv:1208.0928](https://arxiv.org/abs/1208.0928)] and
[[arXiv:1009.3686](https://arxiv.org/abs/1009.3686)]. The surface code for
Majorana qubits is based on
[[arXiv:1909.03002](https://arxiv.org/abs/1909.03002)] and
[[arXiv:2007.00307](https://arxiv.org/abs/2007.00307)] (replacing 8 steps to
measure a single stabilizer in the former reference by 20 steps to measure all
stabilizers). The floquet code, which can only be selected for Majorana qubits,
is based on [[arXiv:2202.11829](https://arxiv.org/abs/2202.11829)].

Pre-defined qubit parameters can be selected by specifying the `name` field in
the `qecScheme` parameter.  If no value is provided, `"surface_code"` is used as
default value.

Let's re-run resource estimation for our running example on the Majorana-based
qubit parameters with a Floquet code.

In [17]:
job = backend.run(circ,
    qubitParams={
        "name": "qubit_maj_ns_e6"
    },
    qecScheme={
        "name": "floquet_code"
    })
result_maj_floquet = job.result()
result_maj_floquet


........

Runtime,6 millisecs,"Total runtime This is a runtime estimate for the execution time of the algorithm. In general, the execution time corresponds to the duration of one logical cycle (900 nanosecs) multiplied by the 6,209 logical cycles to run the algorithm. If however the duration of a single T factory (here: 14,100 nanosecs) is larger than the algorithm runtime, we extend the number of logical cycles artificially in order to exceed the runtime of a single T factory."
rQOPS,50.00M,"Reliable quantum operations per second The value is computed as the number of logical qubits after layout (45) (with a logical error rate of 1.19e-9) multiplied by the clock frequency (1,111,111.11), which is the number of logical cycles per second."
Physical qubits,34.58k,"Number of physical qubits This value represents the total number of physical qubits, which is the sum of 2,340 physical qubits to implement the algorithm logic, and 32,240 physical qubits to execute the T factories that are responsible to produce the T states that are consumed by the algorithm."
Logical algorithmic qubits,45,"Number of logical qubits for the algorithm after layout Laying out the logical qubits in the presence of nearest-neighbor constraints requires additional logical qubits. In particular, to layout the $Q_{\rm alg} = 16$ logical qubits in the input algorithm, we require in total $2 \cdot Q_{\rm alg} + \lceil \sqrt{8 \cdot Q_{\rm alg}}\rceil + 1 = 45$ logical qubits."
Algorithmic depth,6.21k,"Number of logical cycles for the algorithm To execute the algorithm using Parallel Synthesis Sequential Pauli Computation (PSSPC), operations are scheduled in terms of multi-qubit Pauli measurements, for which assume an execution time of one logical cycle. Based on the input algorithm, we require one multi-qubit measurement for the 0 single-qubit measurements, the 702 arbitrary single-qubit rotations, and the 186 T gates, three multi-qubit measurements for each of the 0 CCZ and 0 CCiX gates in the input program, as well as 17 multi-qubit measurements for each of the 313 non-Clifford layers in which there is at least one single-qubit rotation with an arbitrary angle rotation."
Logical depth,6.21k,"Number of logical cycles performed This number is usually equal to the logical depth of the algorithm, which is 6,209. However, in the case in which a single T factory is slower than the execution time of the algorithm, we adjust the logical cycle depth to exceed the T factory's execution time."
Clock frequency,1.11M,Number of logical cycles per second This is the number of logical cycles that can be performed within one second. The logical cycle time is 901 nanosecs.
Number of T states,12.12k,"Number of T states consumed by the algorithm To execute the algorithm, we require one T state for each of the 186 T gates, four T states for each of the 0 CCZ and 0 CCiX gates, as well as 17 for each of the 702 single-qubit rotation gates with arbitrary angle rotation."
Number of T factories,31,"Number of T factories capable of producing the demanded 12,120 T states during the algorithm's runtime The total number of T factories 31 that are executed in parallel is computed as $\left\lceil\dfrac{\text{T states}\cdot\text{T factory duration}}{\text{T states per T factory}\cdot\text{algorithm runtime}}\right\rceil = \left\lceil\dfrac{12,120 \cdot 14,100\;\text{ns}}{1 \cdot 5,588,100\;\text{ns}}\right\rceil$"
Number of T factory invocations,391,"Number of times all T factories are invoked In order to prepare the 12,120 T states, the 31 copies of the T factory are repeatedly invoked 391 times."
Physical algorithmic qubits,2.34k,"Number of physical qubits for the algorithm after layout The 2,340 are the product of the 45 logical qubits after layout and the 52 physical qubits that encode a single logical qubit."


To specify a QEC scheme the user has to specify 2 values, the
`errorCorrectionThreshold` and the `crossingPrefactor`, as well as 2 formulas
for the `logicalCycleTime`, and the `physicalQubitsPerLogicalQubit`.  A template
for QEC schemes is as follows:

```json
{
    "qecScheme": {
        "crossingPrefactor": <double>,
        "errorCorrectionThreshold": <double>,
        "logicalCycleTime": <formula string>,
        "physicalQubitsPerLogicalQubit": <formula string>
    }
}
```

Inside the formulas, the user can make use of the following variables

* `oneQubitGateTime`
* `twoQubitGateTime`
* `oneQubitMeasurementTime`
* `twoQubitJointMeasurementTime`

whose value is taken from the corresponding field from the physical qubit
parameters (note that some variables are not available based on the qubit
parameters' instruction set), as well as the variable

* `codeDistance`

for the code distance computed for the logical qubit, based on the physical
qubit properties, the error correction threshold, and the crossing prefactor.
The time variables and `codeDistance` can be used to describe the
`logicalCycleTime` formula.  For the formula `physicalQubitsPerLogicalQubit`
only the `codeDistance` can be used.

### Error budget

The third parameter `errorBudget` models the total error budget $\epsilon$.  It
sets the overall allowed error for the algorithm, i.e., the number of times it
is allowed to fail.  Its value must be between 0 and 1 and the default value is
0.001, which corresponds to 0.1%, and means that the algorithm is allowed to
fail once in 1000 executions.  This parameter is highly application specific.
For example, if one is running Shor's algorithm for factoring integers, a large
value for the error budget may be tolerated as one can check that the output are
indeed the prime factors of the input.  On the other hand, a much smaller error
budget may be needed for an algorithm solving a problem with a solution which
cannot be efficiently verified.  This budget

$$
  \epsilon = \epsilon_{\log} + \epsilon_{\rm dis} + \epsilon_{\rm syn}
$$

is uniformly distributed and applies to errors $\epsilon_{\log}$ to implement
logical qubits, an error budget $\epsilon_{\rm dis}$ to produce T states through
distillation, and an error budget $\epsilon_{\rm syn}$ to synthesize rotation
gates with arbitrary angles.  Note that for distillation and rotation synthesis,
the respective error budgets $\epsilon_{\rm dis}$ and $\epsilon_{\rm syn}$ are
uniformly distributed among all required T states and all required rotation
gates, respectively. If there are no rotation gates in the input algorithm, the
error budget is uniformly distributed to logical errors and T state errors.

Next, we re-run the last experiment with a an error budget of 10%.

In [ ]:
job = backend.run(circ,
    qubitParams={
        "name": "qubit_maj_ns_e6"
    },
    qecScheme={
        "name": "floquet_code"
    },
    errorBudget=0.1)
result_maj_floquet_e1 = job.result()
result_maj_floquet_e1


## Next steps

We hope you enjoyed this notebook and found it helpful in exploring the physical resource estimates for quantum programs. Here are some suggested next steps:

* Try estimating the resources for a different Qiskit program.
* Explore how qubit parameters and QEC schemes affect the error correction code distance of the logical qubit.
* Visualize your and compare your results from different parameter sets with the space and time diagrams.
* Use the output data to derive logical qubit properties.
* Learn how to setup complex resource estimation experiments in the _Advanced analysis of estimates_ notebook.